In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from openpi.models.pi0_fast_predictor import Pi0FASTPredictor
from openpi.training import config as _config
import openpi.models.model as _model
from openpi.policies import policy_config as _policy_config
from openpi.shared import download
import jax.numpy as jnp
from flax import nnx
import openpi.shared.array_typing as at
import jax
import optax
from flax import nnx, traverse_util
import openpi.training.checkpoints as _checkpoints
import orbax.checkpoint as ocp

from openpi.models.model import restore_params 

/scratch/s5649552/conda/envs/openpi/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
import pathlib

params_path = pathlib.Path(
    "/scratch/s5649552/.cache/openpi/openpi-assets/checkpoints/pi0_fast_libero_predictor/params"
).resolve()
with ocp.PyTreeCheckpointer() as ckptr:
    metadata = ckptr.metadata(params_path)

In [24]:
metadata['params'].keys()

dict_keys(['PaliGemma', '_diffusion_transformer'])

In [6]:
config = _config.get_config("pi0_fast_libero_predictor")
checkpoint_dir = download.maybe_download(
    "/scratch/s5649552/.cache/openpi/openpi-assets/checkpoints/pi0_fast_libero"
)
pali_params = _model.restore_params(checkpoint_dir / "params", dtype=jnp.bfloat16)

In [21]:
config = _config.get_config("pi0_fast_libero_predictor")
checkpoint_dir = download.maybe_download(
    "/scratch/s5649552/.cache/openpi/openpi-assets/checkpoints/pi0_fast_libero_predictor"
)
params = _model.restore_params(checkpoint_dir / "params", dtype=jnp.bfloat16)

In [7]:
model = config.model.create(jax.random.key(0))
graphdef, state = nnx.split(model)

2025-09-01 02:25:35.701324: W external/xla/xla/tsl/framework/bfc_allocator.cc:501] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.96GiB (rounded to 2106589184)requested by op 
2025-09-01 02:25:35.702332: W external/xla/xla/tsl/framework/bfc_allocator.cc:512] *********************************_******************************************************************
E0901 02:25:35.702352 1163968 pjrt_stream_executor_client.cc:3077] Execution of replica 0 failed: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 2106589184 bytes. [tf-allocator-allocation-error='']


ValueError: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 2106589184 bytes.

In [17]:
state.replace_by_pure_dict({"PaliGemma": pali_params["PaliGemma"]})
model = nnx.merge(graphdef, state)
graphdef, state = nnx.split(model)

In [ ]:
graphdef, state = nnx.split(model)
# nnx.display(state)

In [19]:
checkpoint_manager, resuming = _checkpoints.initialize_checkpoint_dir(
        "./checkpoints/pi0_fast_libero_predictor",
        keep_period=config.keep_period,
        overwrite=config.overwrite,
        resume=config.resume,
)

In [20]:
import orbax.checkpoint as ocp
checkpointer = ocp.StandardCheckpointer()
ckpt_dir = ocp.test_utils.erase_and_create_empty('/scratch/s5649552/.cache/openpi/openpi-assets/checkpoints/pi0_fast_libero_predictor')
checkpointer.save(ckpt_dir / 'params', {"params": state})

In [5]:
config.get_freeze_filter()

AttributeError: 'TrainConfig' object has no attribute 'get_freeze_filter'

In [9]:
checkpoint_manager, resuming = _checkpoints.initialize_checkpoint_dir(
    "/scratch/s5649552/openpi/checkpoints/pi0_fast_libero_predictor",
    keep_period=config.keep_period,
    overwrite=config.overwrite,
    resume=config.resume,
)

In [ ]:
params = nnx.state(model)
flat_params = traverse_util.flatten_dict(params, keep_empty_nodes=True)
flat_mask = {k: (k[0] != "PaliGemma") for k in flat_params}  # False for PaliGemma -> frozen
mask = traverse_util.unflatten_dict(flat_mask)

tx = optax.masked(optax.adamw(learning_rate=lr), mask)  # update only non-PaliGemma leaves

train_state = training_utils.TrainState.create(model=model, tx=tx)

manager = ocp.CheckpointManager(
    "/path/to/checkpoints",
    ocp.CheckpointManagerOptions(max_to_keep=3, create=True),
)
ckpt.save_state(manager, train_state, data_loader, step) 

In [ ]:
config = _config.get_config("pi0_fast_libero_predictor")
checkpoint_dir = download.maybe_download(
    "/scratch/s5649552/.cache/openpi/openpi-assets/checkpoints/pi0_fast_libero_predictor"
)
policy = _policy_config.create_trained_policy(config, checkpoint_dir)

In [29]:
params.keys()

dict_keys(['PaliGemma', '_diffusion_transformer'])

In [30]:
import openpi.shared.nnx_utils as nnx_utils

params = nnx.state(model)
# Convert frozen params to bfloat16.
params = nnx_utils.state_map(params, config.freeze_filter, lambda p: p.replace(p.value.astype(jnp.bfloat16)))

AttributeError: 'Pi0FASTPredictor' object has no attribute 'paligemma_variant'

In [34]:
params.filter(config.trainable_filter)['_diffusion_transformer']

State({
  'action_encoder': {
    'fc1': {
      'bias': VariableState(
        type=Param,
        value=Array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)
      ),
      'kernel': VariableState(
        type=Param,
        value=Array([[ 0.31283534, -0.42967808, -0.13735163, ...,  0.12172338,
                 0.4304083 ,  0.80711347],
               [ 0.4414399 ,  0.04935152,  0.6293552 , ...,  0.14129093,
                -0.5374758 , -0.41530365],
               [ 0.11778286, -0.18599501, -0.4302842 , ...,  0.13859908,
                -0.5526887 , -0.23227453],
               ...,
               [-0.58662313, -0.36158082,  0.36350283, ..., -0.34707037,
                 0.3534892 , -0.79183066],
               [-0.27659863, -0.17297661, -0.14692192, ...,  0.7905103 ,
                -0.5905978 , -0.01090948],
               [-0.5167009 , -0.75463194,  0.5624534 , ...,  0.30714113,
                -0.54865944, -0.08579631]], dtype=float32)
      )
    },
    'fc2': {
      'bias': Va

In [1]:
from openpi.training import config
from openpi.policies import policy_config
from openpi.shared import download

config = config.get_config("pi0_fast_libero_predictor")
checkpoint_dir = download.maybe_download("/scratch/s5649552/.cache/openpi/openpi-assets/checkpoints/pi0_fast_libero_predictor")

policy = policy_config.create_trained_policy(config, checkpoint_dir)

/scratch/s5649552/conda/envs/openpi/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ValueError: PyTrees have different structure:
   - at keypath '['_diffusion_transformer']['blocks']': expected <class 'dict'> with pytree metadata (0, 1, 2, 3, 4, 5), got <class 'dict'> with pytree metadata ('0', '1', '2', '3', '4', '5'), so the pytree node metadata does not match.

   - at keypath '['_diffusion_transformer']['video_encoder']['layers']': expected <class 'dict'> with pytree metadata (0, 1, 2, 3, 4, 5, 6, 7), got <class 'dict'> with pytree metadata ('0', '1', '2', '3', '4', '5', '6', '7'), so the pytree node metadata does not match.


In [ ]:
# save
from openpi.models.pi0_fast_predictor import Pi0FASTPredictor
from openpi.training import config as _config
import openpi.models.model as _model
from openpi.policies import policy_config as _policy_config
from openpi.shared import download
import jax.numpy as jnp
from flax import nnx
import openpi.shared.array_typing as at
import jax
import optax
from etils import epath
from flax import nnx, traverse_util
import openpi.training.checkpoints as _checkpoints
import orbax.checkpoint as ocp

from openpi.models.model import restore_params 

config = _config.get_config("pi0_fast_libero_predictor")
checkpoint_dir = download.maybe_download(
    "/scratch/s5649552/.cache/openpi/openpi-assets/checkpoints/pi0_fast_libero"
)
pali_params = _model.restore_params(checkpoint_dir / "params")

model = config.model.create(jax.random.key(0))
graphdef, state = nnx.split(model)

state.replace_by_pure_dict({"PaliGemma": pali_params["PaliGemma"]})
model = nnx.merge(graphdef, state)
graphdef, state = nnx.split(model)

checkpointer = ocp.Checkpointer(ocp.StandardCheckpointHandler())
ckpt_dir = ocp.test_utils.erase_and_create_empty('/scratch/s5649552/.cache/openpi/openpi-assets/checkpoints/pi0_fast_libero_predictor')

/scratch/s5649552/conda/envs/openpi/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
model._diffusion_transformer.blocks

[DiTBlock(
   cross=CrossAttention(
     attn_drop=None,
     dim=1024,
     head_dim=128,
     k=Linear(
       bias=Param(
         value=Array(shape=(1024,), dtype=float32)
       ),
       bias_init=<function zeros at 0x7f67d0f9db20>,
       dot_general=<function dot_general at 0x7f67d1772160>,
       dtype=None,
       in_features=1024,
       kernel=Param(
         value=Array(shape=(1024, 1024), dtype=float32)
       ),
       kernel_init=<function variance_scaling.<locals>.init at 0x7f67d043bf60>,
       out_features=1024,
       param_dtype=<class 'jax.numpy.float32'>,
       precision=None,
       use_bias=True
     ),
     num_heads=8,
     proj=Linear(
       bias=Param(
         value=Array(shape=(1024,), dtype=float32)
       ),
       bias_init=<function zeros at 0x7f67d0f9db20>,
       dot_general=<function dot_general at 0x7f67d1772160>,
       dtype=None,
       in_features=1024,
       kernel=Param(
         value=Array(shape=(1024, 1024), dtype=float32)
       ),
  

In [3]:
params = state.to_pure_dict()

In [8]:
params['_diffusion_transformer']['blocks'].keys()

dict_keys(['0', '1', '2', '3', '4', '5'])

In [5]:
# ensure that all keys and subkeys are strings and if not convert to key
def ensure_str_keys(d):
    if not isinstance(d, dict):
        return d
    return {str(k): ensure_str_keys(v) for k, v in d.items()}
params = ensure_str_keys(params)

In [6]:
# print all keys and subkeys
def print_keys(d, prefix=()):
    if isinstance(d, dict):
        for k, v in d.items():
            path = prefix + (k,)
            print("/".join(map(str, path)))
            print_keys(v, path)
    elif isinstance(d, (list, tuple)):
        for i, v in enumerate(d):
            path = prefix + (f"[{i}]",)
            print("/".join(map(str, path)))
            print_keys(v, path)

print_keys(params)

PaliGemma
PaliGemma/img
PaliGemma/img/Transformer
PaliGemma/img/Transformer/encoder_norm
PaliGemma/img/Transformer/encoder_norm/bias
PaliGemma/img/Transformer/encoder_norm/scale
PaliGemma/img/Transformer/encoderblock
PaliGemma/img/Transformer/encoderblock/LayerNorm_0
PaliGemma/img/Transformer/encoderblock/LayerNorm_0/bias
PaliGemma/img/Transformer/encoderblock/LayerNorm_0/scale
PaliGemma/img/Transformer/encoderblock/LayerNorm_1
PaliGemma/img/Transformer/encoderblock/LayerNorm_1/bias
PaliGemma/img/Transformer/encoderblock/LayerNorm_1/scale
PaliGemma/img/Transformer/encoderblock/MlpBlock_0
PaliGemma/img/Transformer/encoderblock/MlpBlock_0/Dense_0
PaliGemma/img/Transformer/encoderblock/MlpBlock_0/Dense_0/bias
PaliGemma/img/Transformer/encoderblock/MlpBlock_0/Dense_0/kernel
PaliGemma/img/Transformer/encoderblock/MlpBlock_0/Dense_1
PaliGemma/img/Transformer/encoderblock/MlpBlock_0/Dense_1/bias
PaliGemma/img/Transformer/encoderblock/MlpBlock_0/Dense_1/kernel
PaliGemma/img/Transformer/encoder